In [323]:
from copy import deepcopy

In [ ]:
with open("input-example.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [ ]:
initial_field = []
movements = []
first = True
for line in lines:
    if len(line) == 0:
        first = False
    elif first:
        initial_field.append(list(line))
    else:
        movements.append(line)
movements = list("".join(movements))
movements[:5]

In [326]:
# Dir is a number 0 to 3

dirs = {0: (-1, 0), 1: (0, 1), 2: (1, 0), 3: (0, -1)}

dir_map = {"^": 0, ">": 1, "v": 2, "<": 3}


def walk(pos, dir):
    return (pos[0] + dirs[dir][0], pos[1] + dirs[dir][1])


def look_ahead(pos, dir, field):
    next_step = walk(pos, dir)
    if not in_bounds(next_step, field):
        return "."
    return field[next_step[0]][next_step[1]]


def in_bounds(pos, field):
    return (
        pos[0] >= 0 and pos[0] < len(field) and pos[1] >= 0 and pos[1] < len(field[0])
    )


def print_field(field, robot=None):
    i = 0
    for l in field:
        j = 0
        for p in l:
            if robot and robot[0] == i and robot[1] == j:
                print("@", end="")
            else:
                print(p, end="")
            j += 1
        print()
        i += 1

In [ ]:
print_field(initial_field)

In [328]:
def push_block(robot_pos, dir, field) -> tuple[int, int]:
    # find next free field to push to

    # temp pos is stone pos
    temp_pos = walk(robot_pos, dir)
    while True:
        next_field = look_ahead(temp_pos, dir, field)
        if next_field == "#":
            # can't move
            return robot_pos
        elif next_field == ".":
            free_pos = walk(temp_pos, dir)
            field[free_pos[0]][free_pos[1]] = "O"
            first_stone_pos = walk(robot_pos, dir)
            field[first_stone_pos[0]][first_stone_pos[1]] = "."
            return first_stone_pos
        else:
            temp_pos = walk(temp_pos, dir)

In [329]:
field = deepcopy(initial_field)
robot_pos = (0, 0)
for i in range(0, len(field)):
    for j in range(0, len(field[0])):
        if field[i][j] == "@":
            robot_pos = (i, j)
            field[i][j] = "."

for mv in movements:
    dir = dir_map[mv]
    next_field = look_ahead(robot_pos, dir, field)

    # print(robot_pos, mv, next_field)
    if next_field == "#":
        continue
    elif next_field == ".":
        # print("walk")
        robot_pos = walk(robot_pos, dir)
    elif next_field == "O":
        # print("push")
        robot_pos = push_block(robot_pos, dir, field)

In [ ]:
res_sum = 0
for i in range(0, len(field)):
    for j in range(0, len(field[0])):
        if field[i][j] == "O":
            res_sum += 100 * i + j
res_sum

In [ ]:
print_field(field)

In [332]:
# part 2

In [ ]:
field_p2_initial = []
for line in initial_field:
    new_line = []
    for c in line:
        if c == "#":
            new_line += ["#", "#"]
        elif c == "O":
            new_line += ["[", "]"]
        elif c == ".":
            new_line += [".", "."]
        elif c == "@":
            new_line += ["@", "."]
    field_p2_initial.append(new_line)
print_field(field_p2_initial)

In [334]:
def get_box_in_dir(pos, dir, field):
    box_in_dir = {}
    next_field = look_ahead(pos, dir, field)
    if next_field == "[":
        walked_pos = walk(pos, dir)
        box_in_dir[walked_pos] = "["
        box_in_dir[(walked_pos[0], walked_pos[1] + 1)] = "]"
    elif next_field == "]":
        walked_pos = walk(pos, dir)
        box_in_dir[walked_pos] = "]"
        box_in_dir[(walked_pos[0], walked_pos[1] - 1)] = "["

    return box_in_dir


# Could have done recursive pushes or representing everything on the field in a dict for performance reasons.
# But since it was easy and more than fast enough, I move things around the grid of characters
# For a move, the idea is to collect all affected boxes, overwrite them by empty space in the grid and then write the updated boxes with coordinates shifted in the move direction.
def push_block_p2(robot_pos, dir, field) -> tuple[int, int]:
    # 1 is right, 3 is left
    if dir in [1, 3]:
        # left or right can only affect boxes in one line and is very similar to part 1
        temp_pos = robot_pos
        while True:
            next_field = look_ahead(temp_pos, dir, field)
            if next_field == "#":
                # can't move
                return robot_pos
            elif next_field in ["[", "]"]:
                temp_pos = walk(temp_pos, dir)
            elif next_field == ".":
                # temp pos is now the free field, move everything by one field until reaching the robot
                temp_pos = walk(temp_pos, dir)
                if dir == 1:
                    while temp_pos != robot_pos:
                        field[temp_pos[0]][temp_pos[1]] = field[temp_pos[0]][
                            temp_pos[1] - 1
                        ]
                        temp_pos = walk(temp_pos, 3)
                    return walk(robot_pos, 1)
                elif dir == 3:
                    while temp_pos != robot_pos:
                        field[temp_pos[0]][temp_pos[1]] = field[temp_pos[0]][
                            temp_pos[1] + 1
                        ]
                        temp_pos = walk(temp_pos, 1)
                    return walk(robot_pos, 3)
    else:
        # top bottom movement
        # Idea is to check each row of affected boxes
        # if all fields above/below a row of boxes are empty, move
        # if one field is a wall above a box, dont do anything
        # else add boxes above any field of current checked row
        #
        # keep track of boxes in a dict with position and symbol at that position,
        # that way moving all of them is just painting a dot over all existing and re-painting all +1 up/down with symbol
        temp_pos = robot_pos
        all_affected_boxes = {}
        next_line_boxes = {}
        # get initial pushed box
        next_field = look_ahead(temp_pos, dir, field)
        next_line_boxes = get_box_in_dir(temp_pos, dir, field)
        all_affected_boxes.update(next_line_boxes)
        while True:
            # check fields above next line boxes
            next_field_list = []
            for p in next_line_boxes.keys():
                next_field_list.append(look_ahead(p, dir, field))
            if "#" in next_field_list:
                return robot_pos
            elif all([f == "." for f in next_field_list]):
                # Move all boxes
                for p in all_affected_boxes.keys():
                    field[p[0]][p[1]] = "."
                for p in all_affected_boxes.keys():
                    field[walk(p, dir)[0]][walk(p, dir)[1]] = all_affected_boxes[p]
                return walk(robot_pos, dir)
            else:
                # add next row of boxes
                tmp_next_boxes = {}
                for p in next_line_boxes.keys():
                    tmp_next_boxes.update(get_box_in_dir(p, dir, field))
                next_line_boxes = tmp_next_boxes
                all_affected_boxes.update(next_line_boxes)

            pass

In [335]:
field_p2 = deepcopy(field_p2_initial)
robot_pos = (0, 0)
for i in range(0, len(field_p2)):
    for j in range(0, len(field_p2[0])):
        if field_p2[i][j] == "@":
            robot_pos = (i, j)
            field_p2[i][j] = "."

for mv in movements:
    dir = dir_map[mv]
    next_field = look_ahead(robot_pos, dir, field_p2)

    if next_field == "#":
        continue
    elif next_field == ".":
        robot_pos = walk(robot_pos, dir)
    elif next_field in ["[", "]"]:
        robot_pos = push_block_p2(robot_pos, dir, field_p2)

In [ ]:
# as we search from top to bottom and from left to right, check for left box edge only
res_sum = 0
for i in range(0, len(field_p2)):
    for j in range(0, len(field_p2[0])):
        if field_p2[i][j] == "[":
            res_sum += 100 * i + j
res_sum

In [ ]:
print_field(field_p2, robot_pos)